In [ ]:
# %load runSA1SA2.py
import ggcnn.experiment as experiment
import sys
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold


def load_sa1_dataset():
    keys_SA1 = []
    features_SA1 = []
    labels = []
    keys_SA2 = []
    features_SA2 = []
    
    # Load SA1 Node Features
    with open('Data/2018-08-24-NSW-SA1Input-Normalised.csv', 'r') as file:
        for i, line in enumerate(file):
            if i == 0:  # Skip first line (header)
                continue
            s = line[:-1].split(',')  # Last value in line is \n
            keys_SA1.append(s[0])
            features_SA1.extend([float(v) for v in s[1:-1]])  # Last column is the outcome y
#             labels.append(np.floor(float(s[-1]) / 10).astype(int))
            labels.append(float(s[-1]))
    
    
    # Load SA2 Node Features
    with open('Data/2018-08-28-NSW-SA2Input-Normalised.csv', 'r') as file:
        for i, line in enumerate(file):
            if i == 0:  # Skip first line (header)
                continue
            s = line[:-1].split(',')  # Last value in line is \n
            keys_SA2.append(s[0])
            features_SA2.extend([float(v) for v in s[1:-1]])  # Last column is the outcome y

    labels = np.array(labels)
    features_SA1 = np.array(features_SA1).reshape((len(keys_SA1), -1))
    features_SA2 = np.array(features_SA2).reshape((len(keys_SA2), -1))
    
    # Load SA1 Link Features
    with open('Data/2018-08-25-NSW-NeighbourDistance.csv', 'r') as file:
        adj_mat_SA1 = np.zeros((len(keys_SA1), len(keys_SA1)))
        for i, line in enumerate(file):
            if i == 0:  # Skip first line (header)
                continue
            s = line[:-1].split(',')
            a = keys_SA1.index(s[0])
            b = keys_SA1.index(s[1])
            adj_mat_SA1[a, b] = 1
            adj_mat_SA1[b, a] = 1
    

    # Load SA2 Link Features
    with open('Data/Geography/2018-08-28-NSW-SA2_Neighbouring_Suburbs_With_Bridges-GCC.csv', 'r') as file:
        adj_mat_SA2 = np.zeros((len(keys_SA2), len(keys_SA2)))
        for i, line in enumerate(file):
            if i == 0:  # Skip first line (header)
                continue
            s = line[:-1].split(',')
            a = keys_SA2.index(s[0])
            b = keys_SA2.index(s[1])
            adj_mat_SA2[a, b] = 1
            adj_mat_SA2[b, a] = 1
    
    
    # Load SA1, SA2 Links
    with open('Data/SA1SA2Links.csv', 'r') as file:
        adj_mat_SA1SA2 = np.zeros((len(keys_SA1), len(keys_SA2)))
        for i, line in enumerate(file):
            if i == 0:  # Skip first line (header)
                continue
            s = line[:-1].split(',')
            a = keys_SA1.index(s[0])
            b = keys_SA2.index(s[1])
            adj_mat_SA1SA2[a, b] = 1
    
    adj_mat_SA2SA1 = np.transpose(adj_mat_SA1SA2)
    
#     adj_mat_SA1SA2 = adj_mat_SA1SA2 / np.sum(adj_mat_SA1SA2, axis = -1, keepdims = True)
#     adj_mat_SA2SA1 = adj_mat_SA2SA1 / np.sum(adj_mat_SA2SA1, axis = -1, keepdims = True)
    
    return features_SA1, adj_mat_SA1, labels, features_SA2, adj_mat_SA2, adj_mat_SA1SA2, adj_mat_SA2SA1

dataset = load_sa1_dataset()

In [ ]:
class SA1Experiment():
    def __init__(self, neurons, blocks):
        self.blocks = blocks
        self.neurons = neurons
    
    def create_network(self, net, input):
        net.create_network(input)
        
#         net.make_reverse_linkage_adjustment_layer()
        net.make_linkage_adjustment_layer()
        net.make_reverse_auxilary_linkage_layer(None)
#         net.make_auxilary_linkage_layer(0, with_act_func = False)
        
        net.make_embedding_layer(self.neurons)
        net.make_dropout_layer()
        
        for _ in range(self.blocks):
            net.make_graphcnn_layer(self.neurons)
            net.make_dropout_layer()
            net.make_embedding_layer(self.neurons)
            net.make_dropout_layer()
        
#         net.make_auxilary_embedding_layer(self.neurons)
#         net.make_dropout_layer(input_type = 'current_V_auxilary')
#         net.make_reverse_auxilary_linkage_layer(None)
#         net.make_auxilary_embedding_layer(self.neurons)
#         net.make_dropout_layer(input_type = 'current_V_auxilary')
        net.make_auxilary_graphcnn_layer(self.neurons)
        net.make_dropout_layer(input_type = 'current_V_auxilary')
        net.make_auxilary_linkage_layer(None, with_act_func = False)
        
        net.make_embedding_layer(self.neurons)
        net.make_embedding_layer(1, name='final', with_bn=False, with_act_func = False)


no_folds = 5
inst = KFold(n_splits = no_folds, shuffle=True, random_state=125)

l = 2
n = 128
i = 4

    
exp = experiment.GGCNNExperiment('2018-08-28-SA1SA2', '2018-08-28-SA1SA2', SA1Experiment(neurons = n, blocks = l))


exp.num_iterations = 5000
exp.optimizer = 'adam'
exp.loss_type = 'linear'

exp.debug = True  # Was True

exp.preprocess_data(dataset)

# train_idx, test_idx = list(inst.split(np.arange(len(dataset[0]))))[i]
test_idx, train_idx = list(inst.split(np.arange(len(dataset[0]))))[i]  # Reversed to get more samples in the test set than the training set


exp.create_data(train_idx, test_idx, n_components = 3)
exp.build_network()
results = exp.run()

2018-09-01 04:13:18.681765 Creating training Tensorflow Tensors
PCA variance ratio:  [0.16519031 0.1270376  0.0899317 ]
2018-09-01 04:13:18.807920 Creating training network
2018-09-01 04:13:19.852143 Creating loss function and summaries
2018-09-01 04:13:19.900778 Training model "2018-08-28-SA1SA2"!
2018-09-01 04:13:19.901091 Preparing training
2018-09-01 04:13:24.009948 Starting threads
2018-09-01 04:13:24.010145 Starting training. train_batch_size: 0 test_batch_size: 0
2018-09-01 04:13:24.854458 Test Step 0 Finished
2018-09-01 04:13:24.854583 Test Step 0 "min loss" =  1.6366329e+17
2018-09-01 04:13:24.854654 Test Step 0 "loss" =  1.6366329e+17
2018-09-01 04:13:26.984992 Training Step 0 Finished Timing (Training: 0.716368, Test: 0.283539) after 2.97369 seconds
2018-09-01 04:13:26.985146 Training Step 0 "min loss" =  2823.2412
2018-09-01 04:13:26.985968 Training Step 0 "loss" =  2823.2412
2018-09-01 04:13:27.228098 Test Step 5 Finished
2018-09-01 04:13:27.228210 Test Step 5 "min loss" =

2018-09-01 04:13:32.494986 Test Step 95 Finished
2018-09-01 04:13:32.495384 Test Step 95 "min loss" =  1879.5171
2018-09-01 04:13:32.495900 Test Step 95 "loss" =  1879.5171
2018-09-01 04:13:32.549543 Training Step 95 Finished Timing (Training: 0.913323, Test: 0.0808358) after 0.291203 seconds
2018-09-01 04:13:32.550040 Training Step 95 "min loss" =  1931.724
2018-09-01 04:13:32.550454 Training Step 95 "loss" =  1931.724
2018-09-01 04:13:32.788597 Test Step 100 Finished
2018-09-01 04:13:32.788763 Test Step 100 "min loss" =  1849.92
2018-09-01 04:13:32.788840 Test Step 100 "loss" =  1849.92
2018-09-01 04:13:32.843139 Training Step 100 Finished Timing (Training: 0.91334, Test: 0.0808423) after 0.29239 seconds
2018-09-01 04:13:32.843278 Training Step 100 "min loss" =  1893.2754
2018-09-01 04:13:32.843330 Training Step 100 "loss" =  1893.2754
2018-09-01 04:13:33.081697 Test Step 105 Finished
2018-09-01 04:13:33.081830 Test Step 105 "min loss" =  1840.2631
2018-09-01 04:13:33.082502 Test Ste

2018-09-01 04:13:38.336735 Test Step 195 Finished
2018-09-01 04:13:38.337225 Test Step 195 "min loss" =  1091.7406
2018-09-01 04:13:38.337291 Test Step 195 "loss" =  1091.7406
2018-09-01 04:13:38.391227 Training Step 195 Finished Timing (Training: 0.914244, Test: 0.08145) after 0.291539 seconds
2018-09-01 04:13:38.391343 Training Step 195 "min loss" =  1115.4802
2018-09-01 04:13:38.391404 Training Step 195 "loss" =  1115.4802
2018-09-01 04:13:38.630236 Test Step 200 Finished
2018-09-01 04:13:38.630378 Test Step 200 "min loss" =  1045.9357
2018-09-01 04:13:38.630954 Test Step 200 "loss" =  1045.9357
2018-09-01 04:13:38.684547 Training Step 200 Finished Timing (Training: 0.914093, Test: 0.0814172) after 0.292117 seconds
2018-09-01 04:13:38.684820 Training Step 200 "min loss" =  1076.2789
2018-09-01 04:13:38.685325 Training Step 200 "loss" =  1076.2789
2018-09-01 04:13:38.923098 Test Step 205 Finished
2018-09-01 04:13:38.923242 Test Step 205 "min loss" =  1029.8966
2018-09-01 04:13:38.923

2018-09-01 04:13:44.188242 Test Step 295 Finished
2018-09-01 04:13:44.188434 Test Step 295 "min loss" =  476.5008
2018-09-01 04:13:44.188495 Test Step 295 "loss" =  476.5008
2018-09-01 04:13:44.241902 Training Step 295 Finished Timing (Training: 0.914006, Test: 0.0813723) after 0.290768 seconds
2018-09-01 04:13:44.242012 Training Step 295 "min loss" =  472.06808
2018-09-01 04:13:44.242078 Training Step 295 "loss" =  472.06808
2018-09-01 04:13:44.479110 Test Step 300 Finished
2018-09-01 04:13:44.479510 Test Step 300 "min loss" =  459.36285
2018-09-01 04:13:44.479822 Test Step 300 "loss" =  459.36285
2018-09-01 04:13:44.533785 Training Step 300 Finished Timing (Training: 0.91394, Test: 0.0813531) after 0.291643 seconds
2018-09-01 04:13:44.534132 Training Step 300 "min loss" =  446.49228
2018-09-01 04:13:44.534577 Training Step 300 "loss" =  446.49228
2018-09-01 04:13:44.771815 Test Step 305 Finished
2018-09-01 04:13:44.771914 Test Step 305 "min loss" =  423.44022
2018-09-01 04:13:44.7719

2018-09-01 04:13:50.039070 Test Step 395 Finished
2018-09-01 04:13:50.039592 Test Step 395 "min loss" =  160.1837
2018-09-01 04:13:50.039658 Test Step 395 "loss" =  160.1837
2018-09-01 04:13:50.093749 Training Step 395 Finished Timing (Training: 0.913003, Test: 0.0817272) after 0.291115 seconds
2018-09-01 04:13:50.094215 Training Step 395 "min loss" =  143.82361
2018-09-01 04:13:50.094279 Training Step 395 "loss" =  143.82361
2018-09-01 04:13:50.332940 Test Step 400 Finished
2018-09-01 04:13:50.333733 Test Step 400 "min loss" =  155.33879
2018-09-01 04:13:50.333874 Test Step 400 "loss" =  155.33879
2018-09-01 04:13:50.387704 Training Step 400 Finished Timing (Training: 0.912803, Test: 0.0817289) after 0.292914 seconds
2018-09-01 04:13:50.387989 Training Step 400 "min loss" =  133.45825
2018-09-01 04:13:50.388336 Training Step 400 "loss" =  133.45825
2018-09-01 04:13:50.626376 Test Step 405 Finished
2018-09-01 04:13:50.626896 Test Step 405 "min loss" =  155.33879
2018-09-01 04:13:50.627

...

2018-09-01 04:14:58.358778 Test Step 1565 Finished
2018-09-01 04:14:58.358894 Test Step 1565 "min loss" =  7.3428626
2018-09-01 04:14:58.358947 Test Step 1565 "loss" =  7.4647436
2018-09-01 04:14:58.413100 Training Step 1565 Finished Timing (Training: 0.913254, Test: 0.0810052) after 0.290592 seconds
2018-09-01 04:14:58.413200 Training Step 1565 "min loss" =  4.955381
2018-09-01 04:14:58.413258 Training Step 1565 "loss" =  5.6250076
2018-09-01 04:14:58.649874 Test Step 1570 Finished
2018-09-01 04:14:58.650016 Test Step 1570 "min loss" =  7.3428626
2018-09-01 04:14:58.650803 Test Step 1570 "loss" =  7.6054916
2018-09-01 04:14:58.704212 Training Step 1570 Finished Timing (Training: 0.913315, Test: 0.0810389) after 0.290857 seconds
2018-09-01 04:14:58.704302 Training Step 1570 "min loss" =  4.955381
2018-09-01 04:14:58.704344 Training Step 1570 "loss" =  5.9055743
2018-09-01 04:14:58.941070 Test Step 1575 Finished
2018-09-01 04:14:58.941240 Test Step 1575 "min loss" =  7.3428626
2018-09-0

2018-09-01 04:15:03.909280 Test Step 1660 Finished
2018-09-01 04:15:03.909919 Test Step 1660 "min loss" =  7.2424703
2018-09-01 04:15:03.910574 Test Step 1660 "loss" =  7.5095086
2018-09-01 04:15:03.963964 Training Step 1660 Finished Timing (Training: 0.912337, Test: 0.0812082) after 0.291461 seconds
2018-09-01 04:15:03.964420 Training Step 1660 "min loss" =  4.7924104
2018-09-01 04:15:03.964778 Training Step 1660 "loss" =  5.3362074
2018-09-01 04:15:04.201526 Test Step 1665 Finished
2018-09-01 04:15:04.201685 Test Step 1665 "min loss" =  7.2424703
2018-09-01 04:15:04.201752 Test Step 1665 "loss" =  7.6120563
2018-09-01 04:15:04.255059 Training Step 1665 Finished Timing (Training: 0.912339, Test: 0.0812213) after 0.289578 seconds
2018-09-01 04:15:04.255164 Training Step 1665 "min loss" =  4.7751017
2018-09-01 04:15:04.255233 Training Step 1665 "loss" =  5.4315166
2018-09-01 04:15:04.491541 Test Step 1670 Finished
2018-09-01 04:15:04.491677 Test Step 1670 "min loss" =  7.2424703
2018-09

2018-09-01 04:15:09.506082 Test Step 1755 Finished
2018-09-01 04:15:09.506211 Test Step 1755 "min loss" =  7.225133
2018-09-01 04:15:09.506281 Test Step 1755 "loss" =  7.704811
2018-09-01 04:15:09.562023 Training Step 1755 Finished Timing (Training: 0.91381, Test: 0.0813961) after 0.297052 seconds
2018-09-01 04:15:09.562193 Training Step 1755 "min loss" =  4.5928307
2018-09-01 04:15:09.562257 Training Step 1755 "loss" =  4.5928307
2018-09-01 04:15:09.804951 Test Step 1760 Finished
2018-09-01 04:15:09.805146 Test Step 1760 "min loss" =  7.225133
2018-09-01 04:15:09.805219 Test Step 1760 "loss" =  7.72553
2018-09-01 04:15:09.861637 Training Step 1760 Finished Timing (Training: 0.913817, Test: 0.0815997) after 0.299271 seconds
2018-09-01 04:15:09.861796 Training Step 1760 "min loss" =  4.5928307
2018-09-01 04:15:09.861879 Training Step 1760 "loss" =  5.78282
2018-09-01 04:15:10.112021 Test Step 1765 Finished
2018-09-01 04:15:10.112199 Test Step 1765 "min loss" =  7.225133
2018-09-01 04:15

2018-09-01 04:15:15.330451 Test Step 1850 Finished
2018-09-01 04:15:15.330601 Test Step 1850 "min loss" =  7.1115866
2018-09-01 04:15:15.330671 Test Step 1850 "loss" =  7.295466
2018-09-01 04:15:15.386675 Training Step 1850 Finished Timing (Training: 0.913015, Test: 0.0814844) after 0.311232 seconds
2018-09-01 04:15:15.386758 Training Step 1850 "min loss" =  4.5228024
2018-09-01 04:15:15.387190 Training Step 1850 "loss" =  4.942349
2018-09-01 04:15:15.636776 Test Step 1855 Finished
2018-09-01 04:15:15.636968 Test Step 1855 "min loss" =  7.1115866
2018-09-01 04:15:15.637053 Test Step 1855 "loss" =  7.2429533
2018-09-01 04:15:15.693731 Training Step 1855 Finished Timing (Training: 0.913199, Test: 0.0815083) after 0.306463 seconds
2018-09-01 04:15:15.693830 Training Step 1855 "min loss" =  4.5228024
2018-09-01 04:15:15.693897 Training Step 1855 "loss" =  4.8802385
2018-09-01 04:15:15.941941 Test Step 1860 Finished
2018-09-01 04:15:15.942103 Test Step 1860 "min loss" =  7.1115866
2018-09-0

2018-09-01 04:15:21.155514 Test Step 1945 Finished
2018-09-01 04:15:21.156065 Test Step 1945 "min loss" =  7.0203214
2018-09-01 04:15:21.156147 Test Step 1945 "loss" =  7.586166
2018-09-01 04:15:21.213202 Training Step 1945 Finished Timing (Training: 0.913029, Test: 0.0815139) after 0.306019 seconds
2018-09-01 04:15:21.213310 Training Step 1945 "min loss" =  4.341749
2018-09-01 04:15:21.214160 Training Step 1945 "loss" =  4.876022
2018-09-01 04:15:21.464061 Test Step 1950 Finished
2018-09-01 04:15:21.464493 Test Step 1950 "min loss" =  7.0203214
2018-09-01 04:15:21.464804 Test Step 1950 "loss" =  7.4848466
2018-09-01 04:15:21.521294 Training Step 1950 Finished Timing (Training: 0.912732, Test: 0.0815295) after 0.306728 seconds
2018-09-01 04:15:21.521381 Training Step 1950 "min loss" =  4.341749
2018-09-01 04:15:21.521827 Training Step 1950 "loss" =  4.9124494
2018-09-01 04:15:21.771089 Test Step 1955 Finished
2018-09-01 04:15:21.771497 Test Step 1955 "min loss" =  7.0203214
2018-09-01 

In [ ]:
train_df = pd.DataFrame(results[0])
test_df = pd.DataFrame(results[1])
test_df.set_index(test_df.index * exp.iterations_per_test, inplace = True)

In [ ]:
import matplotlib.pyplot as plt

train_df['accuracy'].plot()
test_df['accuracy'].plot()
plt.show()

In [ ]:
train_df['cross_entropy'].loc[10:].plot()
test_df['cross_entropy'].loc[10:].plot()

plt.show()

In [ ]:
import matplotlib.pyplot as plt

train_df['loss'].loc[500:].plot()
test_df['loss'].loc[500:].plot()
train_df['min loss'].loc[500:].plot()
test_df['min loss'].loc[500:].plot()
print(test_df['min loss'].iloc[-1])

plt.show()

In [ ]:
predictions = results[-1].ravel()[test_idx]
actual = dataset[2].ravel()[test_idx]

In [ ]:
plt.scatter(actual, predictions); plt.xlabel('Actual'); plt.ylabel('Predicted'); plt.show()
print('Max error: ', np.max(np.abs(actual - predictions)))

In [ ]:
plt.hist(actual - predictions, bins = 80); plt.show()

In [ ]:
import sklearn

sklearn.metrics.r2_score(predictions, actual)

In [ ]:
np.corrcoef(predictions, actual)